In [9]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [10]:
!pip install -q -U keras-tuner
import kerastuner as kt

# kerastuner를 kt라는 이름으로 설치하고 이용할꺼임

In [11]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

# 옷MNIST

In [12]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# 아까와는 달리 0~1 값으로만 바꿔줌, 정규화
# unit8 -> float32

**모델정의**

In [19]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # inputshape는 28x28인데 flat하게 해주는 레이어 더할거
  
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

    # model.add(tf.keras.layers.Dense(units=32, activation='relu'))
    # model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    # ...
    # model.add(tf.keras.layers.Dense(units=512, activation='relu'))
    # 어느 유닛설정이 가장 성능이 좋은지?
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
    #learning rate = 0.01, 0.001, 0.0001
    # 어느 학습률이 가장 성능이 좋은지?

  return model

In [20]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

# Hyperband 튜닝 알고리즘 사용, 몇 개의 epochs에 대해 많은 수의 모델 훈련 후
# 성능이 좋은 절반만 다음 단계로 넘김

In [21]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)
    
# 에폭이 끝날 때마다 결과 지움

In [22]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

Trial 30 Complete [00h 00m 32s]
val_accuracy: 0.890500009059906

Best val_accuracy So Far: 0.890500009059906
Total elapsed time: 00h 04m 47s
INFO:tensorflow:Oracle triggered exit


In [23]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
# 성능이 제일 좋은 하이퍼파라미터 들을 best_hps로 선언

In [24]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

#최적의 하이퍼파라미터로 훈련해보기

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5910 - accuracy: 0.7924 - val_loss: 0.4006 - val_accuracy: 0.8551
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3673 - accuracy: 0.8677 - val_loss: 0.3880 - val_accuracy: 0.8631
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3327 - accuracy: 0.8780 - val_loss: 0.3575 - val_accuracy: 0.8711
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3023 - accuracy: 0.8890 - val_loss: 0.3814 - val_accuracy: 0.8660
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2858 - accuracy: 0.8947 - val_loss: 0.3344 - val_accuracy: 0.8795
Epoch 6/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2625 - accuracy: 0.9011 - val_loss: 0.3444 - val_accuracy: 0.8791
Epoch 7/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2524 - accuracy: 0.9068 - val_loss: 0.3509 - val_accuracy: